In [2]:
import pandas as pd 

dfNew = pd.read_csv("pv_extraction_results_queue.csv")

In [3]:
dfNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 31 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Title                                         9 non-null      object 
 1   Last Name                                     20 non-null     object 
 2   Year                                          22 non-null     int64  
 3   Digital Object Identifier (DOI)               11 non-null     object 
 4   Research Focus                                22 non-null     object 
 5   Key Findings                                  22 non-null     object 
 6   Device Type                                   22 non-null     object 
 7   Absorber Material                             20 non-null     object 
 8   Absorber Material Term Used                   19 non-null     object 
 9   Absorber Dopant Material                      7 non-null      objec

In [5]:
dfNew.head()

,Title,Last Name,Year,Digital Object Identifier (DOI),Research Focus,Key Findings,Device Type,Absorber Material,Absorber Material Term Used,Absorber Dopant Material,...,Negative Metallization Material,Negative Metallization Material Term Used,Positive Metallization Material,Positive Metallization Material Term Used,Efficiency (%),Cell Area (cm2),Short-Circuit Current (A),Short-Circuit Current Density (mA/cm2),Open-Circuit Voltage (V),Fill Factor (%)
0,Topcon Photovoltaic Cell,Alansary,2024,NaN,Effect of Water Vapor and Humidity on Photovol...,Poor weathering and high humidity have a negat...,Photovoltaic Cell,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Feldmann,2017,NaN,Passivating contacts on silicon solar cells,A 10x10 cm² cell yielded an efficiency of 22.9%.,n-type silicon solar cell,Silicon,Si,Boron,...,NiCuAg,NiCuAg,NaN,NaN,22.9,100.0,NaN,NaN,NaN,NaN
2,NaN,Kafle,2021,10.1016/j.solmat.2021.111100,TOPCon solar cells,TOPCon-concepts are found to remain economical...,Solar Cell,Silicon,poly-Si,N-type,...,NaN,NaN,NaN,NaN,26%,NaN,NaN,NaN,725 mV,NaN
3,Fa-Jun Ma,Ma,2024,NaN,"Silicon solar cells, Space applications, 1 MeV...",When utilizing substrates with high doping lev...,"PERC, TOPCon",Silicon,Silicon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Richtera,2017,10.1016/j.solmat.2017.05.042,Silicon solar cells with passivating contacts,"Recombination induced loss mechanisms, Shockle...",Silicon solar cells,Silicon,c-Si,Boron,...,NaN,NaN,NaN,NaN,25.7%,NaN,42.5,mA/cm2,725 mV,83.3%


In [6]:
dfNew.isna().sum()

Title                                           13
Last Name                                        2
Year                                             0
Digital Object Identifier (DOI)                 11
Research Focus                                   0
Key Findings                                     0
Device Type                                      0
Absorber Material                                2
Absorber Material Term Used                      3
Absorber Dopant Material                        15
Absorber Dopant Material Term Used              15
Absorber Dopant Polarity                        22
Absorber Dopant Polarity Term Used              22
Front Surface Morphology                        20
Front Surface Morphology Term Used              20
Rear Surface Morphology                         22
Rear Surface Morphology Term Used               22
Front Surface Passivation Material              15
Front Surface Passivation Material Term Used    16
Rear Surface Passivation Materi

In [2]:
# --- Imports ---
import json
import pandas as pd
from pathlib import Path
from pydantic import BaseModel, Field, validator
from tqdm import tqdm
from langchain_community.document_loaders import PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

# --- Constants ---
PDF_FOLDER = Path("foldersam")
MODEL_NAME = "gemma3:1b"
OUTPUT_CSV = "pv_extraction_results_ollama.csv"
CHUNK_SIZE = 2000
CHUNK_OVERLAP = 200

PROMPT_TEMPLATE = """
You are extracting structured data from academic articles on photovoltaic cells.
Focus only on the most efficient cell mentioned in each article.

IMPORTANT:
- Return ALL fields from the schema.
- If any field is not available, set its value to "N/A".
- Return ONLY a single JSON object.
- Do not use markdown or code formatting.

Article:
{text}

Format like:
{format_instructions}
"""

COLUMN_MAP = {
    "title": "Title", "last_name": "Last Name", "year": "Year", "doi": "Digital Object Identifier (DOI)",
    "research_focus": "Research Focus", "key_findings": "Key Findings", "device_type": "Device Type",
    "absorber_material": "Absorber Material", "absorber_material_term_used": "Absorber Material Term Used",
    "absorber_dopant_material": "Absorber Dopant Material", "absorber_dopant_material_term_used": "Absorber Dopant Material Term Used",
    "absorber_dopant_polarity": "Absorber Dopant Polarity", "absorber_dopant_polarity_term_used": "Absorber Dopant Polarity Term Used",
    "front_surface_morphology": "Front Surface Morphology", "front_surface_morphology_term_used": "Front Surface Morphology Term Used",
    "rear_surface_morphology": "Rear Surface Morphology", "rear_surface_morphology_term_used": "Rear Surface Morphology Term Used",
    "front_surface_passivation_material": "Front Surface Passivation Material", "front_surface_passivation_material_term_used": "Front Surface Passivation Material Term Used",
    "rear_surface_passivation_material": "Rear Surface Passivation Material", "rear_surface_passivation_material_term_used": "Rear Surface Passivation Material Term Used",
    "negative_metallization_material": "Negative Metallization Material", "negative_metallization_material_term_used": "Negative Metallization Material Term Used",
    "positive_metallization_material": "Positive Metallization Material", "positive_metallization_material_term_used": "Positive Metallization Material Term Used",
    "efficiency_percent": "Efficiency (%)", "cell_area_cm2": "Cell Area (cm2)",
    "short_circuit_current_a": "Short-Circuit Current (A)", "short_circuit_current_density_ma_cm2": "Short-Circuit Current Density (mA/cm2)",
    "open_circuit_voltage_v": "Open-Circuit Voltage (V)", "fill_factor_percent": "Fill Factor (%)"
}

# --- Data Schema ---
class PVArticleData(BaseModel):
    title: str = Field("N/A")
    last_name: str = Field("N/A")
    year: str = Field("N/A")
    doi: str = Field("N/A")
    research_focus: str = Field("N/A")
    key_findings: str = Field("N/A")
    device_type: str = Field("N/A")
    absorber_material: str = Field("N/A")
    absorber_material_term_used: str = Field("N/A")
    absorber_dopant_material: str = Field("N/A")
    absorber_dopant_material_term_used: str = Field("N/A")
    absorber_dopant_polarity: str = Field("N/A")
    absorber_dopant_polarity_term_used: str = Field("N/A")
    front_surface_morphology: str = Field("N/A")
    front_surface_morphology_term_used: str = Field("N/A")
    rear_surface_morphology: str = Field("N/A")
    rear_surface_morphology_term_used: str = Field("N/A")
    front_surface_passivation_material: str = Field("N/A")
    front_surface_passivation_material_term_used: str = Field("N/A")
    rear_surface_passivation_material: str = Field("N/A")
    rear_surface_passivation_material_term_used: str = Field("N/A")
    negative_metallization_material: str = Field("N/A")
    negative_metallization_material_term_used: str = Field("N/A")
    positive_metallization_material: str = Field("N/A")
    positive_metallization_material_term_used: str = Field("N/A")
    efficiency_percent: str = Field("N/A")
    cell_area_cm2: str = Field("N/A")
    short_circuit_current_a: str = Field("N/A")
    short_circuit_current_density_ma_cm2: str = Field("N/A")
    open_circuit_voltage_v: str = Field("N/A")
    fill_factor_percent: str = Field("N/A")

    @validator("*", pre=True)
    def convert_to_string(cls, v):
        return "N/A" if v is None else str(v)

# --- Helper Functions ---
def clean_response_data(response_data):
    if isinstance(response_data, list):
        response_data = response_data[0] if response_data else {}
    if isinstance(response_data, PVArticleData):
        return response_data.model_dump()
    if isinstance(response_data, str):
        cleaned = response_data.strip().removeprefix("```json").removeprefix("```").removesuffix("```").strip()
        try:
            parsed = json.loads(cleaned)
            return parsed[0] if isinstance(parsed, list) else parsed
        except json.JSONDecodeError:
            return {}
    if isinstance(response_data, dict):
        return response_data
    return {}

def count_filled_fields(data: dict) -> int:
    return sum(1 for v in data.values() if v != "N/A")

# --- Main Script ---
def main():
    model = ChatOllama(model=MODEL_NAME)
    parser = PydanticOutputParser(pydantic_object=PVArticleData)
    prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)
    chain = prompt | model | parser

    pdf_files = list(PDF_FOLDER.rglob("*.pdf"))
    results = []

    for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
        try:
            loader = PDFMinerLoader(str(pdf_file))
            docs = loader.load()
            full_text = "\n\n".join([doc.page_content for doc in docs])
            splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
            chunks = splitter.split_text(full_text)

            best_data = None
            highest_filled_count = 0
            for chunk in chunks:
                try:
                    response = chain.invoke({
                        "text": chunk,
                        "format_instructions": parser.get_format_instructions()
                    })
                    cleaned_data = clean_response_data(response)
                    if cleaned_data:
                        filled_count = count_filled_fields(cleaned_data)
                        if filled_count > highest_filled_count:
                            highest_filled_count = filled_count
                            best_data = cleaned_data
                except Exception:
                    continue

            if best_data:
                article_data = PVArticleData(**best_data)
                results.append(article_data.model_dump())
        except Exception:
            continue

    if results:
        df = pd.DataFrame(results)
        df = df.rename(columns=COLUMN_MAP)
        df = df[list(COLUMN_MAP.values())]
        df.to_csv(OUTPUT_CSV, index=False)
        print(f"\n✅ Saved the CSV to {OUTPUT_CSV}")
    else:
        print("\n⚠️ No results to save.")

# --- Run ---
if __name__ == "__main__":
    main()


C:\Users\bootcamp\AppData\Local\Temp\ipykernel_4008\3053518829.py:88: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("*", pre=True)
Processing PDFs: 100%|██████████| 1/1 [45:29<00:00, 2729.63s/it]


⚠️ No results to save.
